In [6]:
# 1. Install and import necessary packages
!pip install gradio transformers



In [7]:

import gradio as gr
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
# Set the random seed
torch.manual_seed(0)

# 2. Load the tokenizer and add special tokens
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# 3. Load the pretrained model
model = GPT2LMHeadModel.from_pretrained('distilgpt2')

# Resize token embeddings in case of added tokens
model.resize_token_embeddings(len(tokenizer))

# 4. Load the fine-tuned weights into the model
model_save_name = "fine_tuned_gpt2.pt"
model.load_state_dict(torch.load(model_save_name))
model.eval()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50258, bias=False)
)

In [8]:
# 4. Define the generation function
def generate_story(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    max_length = len(input_ids[0]) + 100
    sample_output = model.generate(input_ids, max_length=max_length, num_beams=5, top_k=50, temperature=0.7, pad_token_id=tokenizer.eos_token_id)
    story = tokenizer.decode(sample_output[0], skip_special_tokens=True)
    return story

# 5. Define and launch the Gradio interface
interface = gr.Interface(
    fn=generate_story, 
    inputs=gr.inputs.Textbox(lines=5, placeholder="Enter your prompt here..."), 
    outputs=gr.outputs.Textbox(),
    live=True
)
interface.launch()

C:\Users\HP\AppData\Local\Temp\ipykernel_1744\2881837736.py:12: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Textbox(lines=5, placeholder="Enter your prompt here..."),
C:\Users\HP\AppData\Local\Temp\ipykernel_1744\2881837736.py:12: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=5, placeholder="Enter your prompt here..."),
C:\Users\HP\AppData\Local\Temp\ipykernel_1744\2881837736.py:12: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Textbox(lines=5, placeholder="Enter your prompt here..."),
C:\Users\HP\AppData\Local\Temp\ipykernel_1744\2881837736.py:13: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Textbox(

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
